In [20]:
# -*- coding: utf-8 -*-
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu
from geoloc_functions import distance_to_centre, direction_from_centre
from openlocationcode import openlocationcode as olc
from pluscode_functions import convert_dic, convert_pluscode
from iris_functions import create_iris_polygon_dict, find_iris

In [2]:
# Read recipe inputs
v75_2014_2021_transactions_partial_pluscode = dataiku.Dataset("75_2014_2021_transactions_partial_pluscode")
v75_2014_2021_transactions_partial_pluscode_df = v75_2014_2021_transactions_partial_pluscode.get_dataframe()
iris_75 = dataiku.Dataset("iris_75")
iris_75_df = iris_75.get_dataframe()

/Users/nathanbry/Dataiku/dss-design/code-envs/python/real_estate/lib/python3.7/site-packages/pandas/io/parsers.py:2037: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  data = self._reader.read(nrows)
/Users/nathanbry/Dataiku/dss-design/code-envs/python/real_estate/lib/python3.7/site-packages/pandas/core/indexes/base.py:395: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  elif issubclass(data.dtype.type, np.bool) or

In [3]:
v75_2014_2021_transactions_partial_pluscode_df['distance_to_centre']= distance_to_centre(v75_2014_2021_transactions_partial_pluscode_df['latitude'],v75_2014_2021_transactions_partial_pluscode_df['longitude'])

In [4]:
v75_2014_2021_transactions_partial_pluscode_df['direction_from_centre'] = np.vectorize(direction_from_centre)(v75_2014_2021_transactions_partial_pluscode_df['longitude'],v75_2014_2021_transactions_partial_pluscode_df['latitude'])

In [5]:
v75_2014_2021_transactions_partial_pluscode_df

,date_mutation,date_mutation_year,date_mutation_month,date_mutation_day,longitude,latitude,type_local,valeur_fonciere_avg,code_postal,lot1_surface_carrez_sum,surface_reelle_bati_sum,surface_m2_not_null,prix_m2_not_null,nombre_pieces_principales_sum,GeoPoint_transaction,pluscode_16_first,distance_to_centre,direction_from_centre
0,2014-04-30T00:00:00.000Z,2014,4,30,2.276194,48.863192,Appartement,819400.0,75016,75.73,75.0,75.73,10820.018487,4.0,POINT(2.276194 48.863192),8FW4V77G+7FGJ9VVR,4.901700,W
1,2014-04-30T00:00:00.000Z,2014,4,30,2.276194,48.863192,Dépendance,819400.0,75016,75.73,NaN,75.73,10820.018487,0.0,POINT(2.276194 48.863192),8FW4V77G+7FGJ9VVR,4.901700,W
2,2014-04-30T00:00:00.000Z,2014,4,30,2.276761,48.844500,Appartement,410000.0,75015,45.72,46.0,45.72,8967.629046,2.0,POINT(2.276761 48.8445),8FW4R7VG+QPRVVWWR,4.986228,W
3,2014-04-30T00:00:00.000Z,2014,4,30,2.280347,48.836862,Appartement,449000.0,75015,NaN,49.0,49.00,9163.265306,2.0,POINT(2.280347 48.836862),8FW4R7PJ+P4XC7WWW,5.040071,W
4,2014-04-30T00:00:00.000Z,2014,4,30,2.280998,48.881305,Appartement,362000.0,75017,NaN,69.0,69.00,5246.376812,3.0,POINT(2.280998 48.881305),8FW4V7JJ+G9H5WXXW,5.272083,W
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
289481,2014-04-30T00:00:00.000Z,2014,4,30,2.269628,48.850294,Appartement,150000.0,75016,NaN,33.0,33.00,4545.454545,1.0,POINT(2.269628 48.850294),8FW4V729+4V6JMWRW,5.370576,W
289482,2014-04-30T00:00:00.000Z,2014,4,30,2.272693,48.851675,Appartement,150000.0,75016,NaN,17.0,17.00,8823.529412,1.0,POINT(2.272693 48.851675),8FW4V72F+M3G24532,5.129789,W
289483,2014-04-30T00:00:00.000Z,2014,4,30,2.272854,48.856130,Appartement,195000.0,75016,21.53,21.0,21.53,9057.129587,1.0,POINT(2.272854 48.85613),8FW4V74F+F4573255,5.089795,W
289484,2014-04-30T00:00:00.000Z,2014,4,30,2.273469,48.847895,Appartement,522500.0,75016,NaN,56.0,56.00,9330.357143,1.0,POINT(2.273469 48.847895),8FW4R7XF+595JRRWR,5.135025,W


In [6]:
data_encoded = v75_2014_2021_transactions_partial_pluscode_df[v75_2014_2021_transactions_partial_pluscode_df['pluscode_16_first'].isnull() == False]
data_to_encode = v75_2014_2021_transactions_partial_pluscode_df[v75_2014_2021_transactions_partial_pluscode_df['pluscode_16_first'].isnull() == True]

In [7]:
data_to_encode['pluscode_16_first'] = np.vectorize(olc.encode)(data_to_encode['latitude'],data_to_encode['longitude'],17)

/Users/nathanbry/Dataiku/dss-design/code-envs/python/real_estate/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [8]:
full_data = pd.concat([data_encoded,data_to_encode])

In [9]:
full_data['pluscode_10'] = [code[:11] for code in full_data['pluscode_16_first'].to_list()]

/Users/nathanbry/Dataiku/dss-design/code-envs/python/real_estate/lib/python3.7/site-packages/pandas/core/indexes/base.py:395: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  elif issubclass(data.dtype.type, np.bool) or is_bool_dtype(data):


In [10]:
full_data['v_coordinate'],full_data['h_coordinate'],full_data['reduced_v_coordinate'],full_data['reduced_h_coordinate'],full_data['reduced_coordinates_couple'] = convert_pluscode(full_data['pluscode_10'])

In [11]:
iris_75_df

,Geo Point,Geo Shape,DEP,INSEE_COM,NOM_COM,IRIS,CODE_IRIS,NOM_IRIS,TYP_IRIS
0,"48.8636247103,2.34626511981","{""type"": ""Polygon"", ""coordinates"": [[[2.349615...",75,75101,Paris 1er Arrondissement,204,751010204,Les Halles 4,H
1,"48.8595579839,2.34179582076","{""type"": ""Polygon"", ""coordinates"": [[[2.339862...",75,75101,Paris 1er Arrondissement,103,751010103,Saint-Germain l'Auxerrois 3,A
2,"48.8629668422,2.33561969841","{""type"": ""Polygon"", ""coordinates"": [[[2.339468...",75,75101,Paris 1er Arrondissement,303,751010303,Palais Royal 3,A
3,"48.8636973438,2.32666764377","{""type"": ""Polygon"", ""coordinates"": [[[2.332074...",75,75101,Paris 1er Arrondissement,105,751010105,Tuileries,D
4,"48.8589854156,2.33620766992","{""type"": ""Polygon"", ""coordinates"": [[[2.339862...",75,75101,Paris 1er Arrondissement,199,751010199,Seine et Berges,D
...,...,...,...,...,...,...,...,...,...
987,"48.8742331805,2.38975913839","{""type"": ""Polygon"", ""coordinates"": [[[2.389556...",75,75120,Paris 20e Arrondissement,7712,751207712,Belleville 12,H
988,"48.8703700882,2.40165818488","{""type"": ""Polygon"", ""coordinates"": [[[2.402367...",75,75120,Paris 20e Arrondissement,7808,751207808,Saint-Fargeau 8,H
989,"48.8683425761,2.39761116833","{""type"": ""Polygon"", ""coordinates"": [[[2.394468...",75,75120,Paris 20e Arrondissement,7915,751207915,Pere Lachaise 15,H
990,"48.8634483283,2.39830798479","{""type"": ""Polygon"", ""coordinates"": [[[2.400224...",75,75120,Paris 20e Arrondissement,7904,751207904,Pere Lachaise 4,H


In [12]:
iris_coords_codes = create_iris_polygon_dict(iris_75_df['CODE_IRIS'],iris_75_df['Geo Shape'])

In [24]:
def find_iris(latitude_column, longitude_column, iris_coords_codes_dict):
    lat_list = latitude_column.to_list()
    long_list = longitude_column.to_list()

    points_list = []

    for i in range(len(lat_list)):
        coords_point = Point(long_list[i],lat_list[i])

        points_list.append(coords_point)
        
        
    points_iris_code_list = []

    for point in points_list:

        is_within_polygon = False

        for iris,coords in iris_coords_codes_dict.items():

            test = point.within(coords)

            if test == True:

                points_iris_code_list.append(iris)

                is_within_polygon = True

                break

        if is_within_polygon==False:

            points_iris_code_list.append('None')
            
    return points_iris_code_list

In [25]:
points_iris_code = find_iris(full_data['latitude'],full_data['longitude'],iris_coords_codes)

In [27]:
len(points_iris_code)

289486

In [28]:
lat_list = full_data['latitude'].to_list()
long_list = full_data['longitude'].to_list()

points_list = []

for i in range(len(lat_list)):
    coords_point = Point(long_list[i],lat_list[i])
   
    points_list.append(coords_point)

In [29]:
points_iris_code = []


for point in points_list:
     
    bbb = False
       
    for iris,coords in iris_coords_codes.items():
       
        test = point.within(coords)

#         print(test)

        if test == True:

            points_iris_code.append(iris)
           
            bbb = True

#             print(iris)
           
            break
       
    if bbb==False:
       
        points_iris_code.append('None')


In [30]:
len(points_iris_code)

289486

In [32]:
points_iris_code.count('None')

62

In [33]:
full_data['iris_code'] = points_iris_code

/Users/nathanbry/Dataiku/dss-design/code-envs/python/real_estate/lib/python3.7/site-packages/pandas/core/indexes/base.py:395: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  elif issubclass(data.dtype.type, np.bool) or is_bool_dtype(data):


In [34]:
full_data_enriched = pd.merge(full_data, iris_75_df,left_on='iris_code',right_on='CODE_IRIS',how='inner')

In [35]:
full_data_enriched

,date_mutation,date_mutation_year,date_mutation_month,date_mutation_day,longitude,latitude,type_local,valeur_fonciere_avg,code_postal,lot1_surface_carrez_sum,surface_reelle_bati_sum,surface_m2_not_null,prix_m2_not_null,nombre_pieces_principales_sum,GeoPoint_transaction,pluscode_16_first,distance_to_centre,direction_from_centre,pluscode_10,v_coordinate,h_coordinate,reduced_v_coordinate,reduced_h_coordinate,reduced_coordinates_couple,iris_code,Geo Point,Geo Shape,DEP,INSEE_COM,NOM_COM,IRIS,CODE_IRIS,NOM_IRIS,TYP_IRIS
0,2014-04-30T00:00:00.000Z,2014,4,30,2.276194,48.863192,Appartement,819400.0,75016,75.73,75.0,75.73,10820.018487,4.0,POINT(2.276194 48.863192),8FW4V77G+7FGJ9VVR,4.901700,W,8FW4V77G+7F,25,10,12,5,"12,5",751166209,"48.8632739301,2.27468458749","{""type"": ""Polygon"", ""coordinates"": [[[2.277430...",75,75116,Paris 16e Arrondissement,6209,751166209,Muette 9,H
1,2014-04-30T00:00:00.000Z,2014,4,30,2.276194,48.863192,Dépendance,819400.0,75016,75.73,NaN,75.73,10820.018487,0.0,POINT(2.276194 48.863192),8FW4V77G+7FGJ9VVR,4.901700,W,8FW4V77G+7F,25,10,12,5,"12,5",751166209,"48.8632739301,2.27468458749","{""type"": ""Polygon"", ""coordinates"": [[[2.277430...",75,75116,Paris 16e Arrondissement,6209,751166209,Muette 9,H
2,2014-05-05T00:00:00.000Z,2014,5,5,2.275868,48.862550,Local industriel. commercial ou assimilé,120000.0,75016,25.28,22.0,25.28,4746.835443,0.0,POINT(2.275868 48.86255),8FW4V77G+289XRVWW,4.915026,W,8FW4V77G+28,25,10,12,5,"12,5",751166209,"48.8632739301,2.27468458749","{""type"": ""Polygon"", ""coordinates"": [[[2.277430...",75,75116,Paris 16e Arrondissement,6209,751166209,Muette 9,H
3,2014-05-09T00:00:00.000Z,2014,5,9,2.274707,48.863452,Dépendance,45000.0,75016,7.70,NaN,7.70,5844.155844,0.0,POINT(2.274707 48.863452),8FW4V77F+9VP47XXW,5.013647,W,8FW4V77F+9V,25,9,12,4,"12,4",751166209,"48.8632739301,2.27468458749","{""type"": ""Polygon"", ""coordinates"": [[[2.277430...",75,75116,Paris 16e Arrondissement,6209,751166209,Muette 9,H
4,2014-05-19T00:00:00.000Z,2014,5,19,2.274673,48.862905,Appartement,979200.0,75016,108.01,89.0,108.01,9065.827238,3.0,POINT(2.274673 48.862905),8FW4V77F+5V782432,5.007114,W,8FW4V77F+5V,25,9,12,4,"12,4",751166209,"48.8632739301,2.27468458749","{""type"": ""Polygon"", ""coordinates"": [[[2.277430...",75,75116,Paris 16e Arrondissement,6209,751166209,Muette 9,H
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
289419,2015-07-31T00:00:00.000Z,2015,7,31,2.271883,48.839679,Local industriel. commercial ou assimilé,95598750.0,75015,NaN,2209.0,2209.00,43276.935265,0.0,POINT(2.271883 48.839679),8FW4R7QC+VQC3R23,5.490182,W,8FW4R7QC+VQ,15,8,7,4,"7,4",751156021,"48.8381820541,2.27349732987","{""type"": ""Polygon"", ""coordinates"": [[[2.278203...",75,75115,Paris 15e Arrondissement,6021,751156021,Javel 21,A
289420,2016-12-16T00:00:00.000Z,2016,12,16,2.376643,48.821016,Appartement,133852728.0,75013,NaN,440.0,440.00,304210.745455,21.0,POINT(2.376643 48.821016),8FW4R9CG+CM2P725,4.675728,SE,8FW4R9CG+CM,8,50,4,25,"4,25",751135001,"48.8221592584,2.37604291406","{""type"": ""Polygon"", ""coordinates"": [[[2.375305...",75,75113,Paris 13e Arrondissement,5001,751135001,Gare 1,H
289421,2016-12-20T00:00:00.000Z,2016,12,20,2.344876,48.900722,Appartement,2521500.0,75018,NaN,252.0,252.00,10005.952381,14.0,POINT(2.344876 48.900722),8FW4W82V+7XJRC42,4.922139,N,8FW4W82V+7X,40,37,20,18,"20,18",751187027,"48.8998156449,2.34810324436","{""type"": ""Polygon"", ""coordinates"": [[[2.351873...",75,75118,Paris 18e Arrondissement,7027,751187027,Clignancourt 27,H
289422,2018-12-18T00:00:00.000Z,2018,12,18,2.368869,48.884758,Local industriel. commercial ou assimilé,33260000.0,75019,NaN,2156.0,2156.00,15426.716141,0.0,POINT(2.368869 48.884758),8FW4V9M9+WG59J54,3.691813,NE,8FW4V9M9+WG,33,47,16,23,"16,23",751197318,"48.8853543615,2.3692362348","{""type"": ""Polygon"", ""coordinates"": [[[2.369133...",75,75119,Paris 19e Arrondissement,7318,751197

In [0]:
# Compute recipe outputs
# TODO: Write here your actual code that computes the outputs
# NB: DSS supports several kinds of APIs for reading and writing data. Please see doc.

transactions_enriched_df = full_data_enriched # Compute a Pandas dataframe to write into transactions_enriched


# Write recipe outputs
transactions_enriched = dataiku.Dataset("transactions_enriched")
transactions_enriched.write_with_schema(transactions_enriched_df)